# Camera Calibration wit OpenCV
https://docs.opencv.org/4.5.3/dc/dbb/tutorial_py_calibration.html

In [1]:
import numpy as np
import cv2 as cv
import glob
# termination criteria

# Inner chessboard size (in number of squares)
cbSize = [7,9]
pix_mm = 217/10

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((cbSize[0]*cbSize[1],3), np.float32)
objp[:,:2] = np.mgrid[0:cbSize[1],0:cbSize[0]].T.reshape(-1,2)*pix_mm

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob("./data/finse_webcam/*.jpg")
images.sort()

for fname in images:
    print(fname)
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (cbSize[1],cbSize[0]), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (cbSize[1],cbSize[0]), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

./data/finse_webcam/129.240.244.154_01_20220330140418935_TIMING.jpg
./data/finse_webcam/129.240.244.154_01_20220330140423935_TIMING.jpg
./data/finse_webcam/129.240.244.154_01_20220330140428935_TIMING.jpg
./data/finse_webcam/129.240.244.154_01_20220330140433953_TIMING.jpg
./data/finse_webcam/129.240.244.154_01_20220330140439010_TIMING.jpg
./data/finse_webcam/129.240.244.154_01_20220330140444003_TIMING.jpg
./data/finse_webcam/129.240.244.154_01_20220330140449002_TIMING.jpg
./data/finse_webcam/129.240.244.154_01_20220330140454002_TIMING.jpg
./data/finse_webcam/129.240.244.154_01_20220330140514024_TIMING.jpg
./data/finse_webcam/129.240.244.154_01_20220330140519055_TIMING.jpg
./data/finse_webcam/129.240.244.154_01_20220330140524056_TIMING.jpg
./data/finse_webcam/129.240.244.154_01_20220330140529055_TIMING.jpg
./data/finse_webcam/129.240.244.154_01_20220330140534063_TIMING.jpg
./data/finse_webcam/129.240.244.154_01_20220330140539095_TIMING.jpg
./data/finse_webcam/129.240.244.154_01_202203301

In [2]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [3]:
mtx

array([[1.46143486e+03, 0.00000000e+00, 9.89097728e+02],
       [0.00000000e+00, 1.45901969e+03, 5.17957369e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [4]:
img = cv.imread(images[0])
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [5]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult_webcam.png', dst)

True

In [6]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.18021223269168568


In [7]:
dict(zip(['k1','k2','p1','p2','k3'],dist[0]))

{'k1': -0.4200893485833154,
 'k2': 0.31414876662568736,
 'p1': 0.007246482749990121,
 'p2': 0.0008494067035016822,
 'k3': -0.16517855431212075}

In [8]:
dist

array([[-0.42008935,  0.31414877,  0.00724648,  0.00084941, -0.16517855]])